<a href="https://colab.research.google.com/github/anhbui5302/nanoGPT/blob/master/examples/language_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [ ]:
# Install required libraries
!pip install -q -U transformers datasets accelerate peft bitsandbytes scikit-learn

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Then you need to install Git-LFS. Uncomment the following instructions:

In [ ]:
!apt install git-lfs

Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [ ]:
import transformers

print(transformers.__version__)

You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/language-modeling).

We also quickly upload some telemetry - this tells us which examples and software versions are getting used so we know where to prioritize our maintenance efforts. We don't collect (or care about) any personally identifiable information, but if you'd prefer not to be counted, feel free to skip this step or delete this cell entirely.

In [ ]:
from transformers.utils import send_example_telemetry

send_example_telemetry("language_modeling_notebook", framework="pytorch")

# Fine-tuning a language model

In this notebook, we'll see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model on a language modeling tasks. We will cover two types of language modeling tasks which are:

- Causal language modeling: the model has to predict the next token in the sentence (so the labels are the same as the inputs shifted to the right). To make sure the model does not cheat, it gets an attention mask that will prevent it to access the tokens after token i when trying to predict the token i+1 in the sentence.

![Widget inference representing the causal language modeling task](https://github.com/huggingface/notebooks/blob/master/examples/images/causal_language_modeling.png?raw=1)

- Masked language modeling: the model has to predict some tokens that are masked in the input. It still has access to the whole sentence, so it can use the tokens before and after the tokens masked to predict their value.

![Widget inference representing the masked language modeling task](https://github.com/huggingface/notebooks/blob/master/examples/images/masked_language_modeling.png?raw=1)

We will see how to easily load and preprocess the dataset for each one of those tasks, and how to use the `Trainer` API to fine-tune a model on it.

A script version of this notebook you can directly run on a distributed environment or on TPU is available in our [examples folder](https://github.com/huggingface/transformers/tree/master/examples).

## Preparing the dataset

For each of those tasks, we will use the [Wikitext 2]() dataset as an example. You can load it very easily with the 🤗 Datasets library.

In [ ]:
from datasets import load_dataset
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')

You can replace the dataset above with any dataset hosted on [the hub](https://huggingface.co/datasets) or use your own files. Just uncomment the following cell and replace the paths with values that will lead to your files:

In [ ]:
# datasets = load_dataset("text", data_files={"train": path_to_train.txt, "validation": path_to_validation.txt}

You can also load datasets from a csv or a JSON file, see the [full documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) for more information.

To access an actual element, you need to select a split first, then give an index:

In [ ]:
datasets["train"][10]

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

As we can see, some of the texts are a full paragraph of a Wikipedia article while others are just titles or empty lines.

## Causal Language modeling

For causal language modeling (CLM) we are going to take all the texts in our dataset and concatenate them after they are tokenized. Then we will split them in examples of a certain sequence length. This way the model will receive chunks of contiguous text that may look like:
```
part of text 1
```
or
```
end of text 1 [BOS_TOKEN] beginning of text 2
```
depending on whether they span over several of the original texts in the dataset or not. The labels will be the same as the inputs, shifted to the left.

We will use the [`distilgpt2`](https://huggingface.co/distilgpt2) model for this example. You can pick any of the checkpoints listed [here](https://huggingface.co/models?filter=causal-lm) instead:

In [ ]:
model_checkpoint = "google/gemma-3-1b-it"

To tokenize all our texts with the same vocabulary that was used when training the model, we have to download a pretrained tokenizer. This is all done by the `AutoTokenizer` class:

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


In [ ]:
tokenizer.pad_token

We can now call the tokenizer on all our texts. This is very simple, using the [`map`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map) method from the Datasets library. First we define a function that call the tokenizer on our texts:

In [ ]:
def tokenize_function(examples):
    # If the tokenizer doesn't have a pad token, add one.
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    return tokenizer(examples["text"])

Then we apply it to all the splits in our `datasets` object, using `batched=True` and 4 processes to speed up the preprocessing. We won't need the `text` column afterward, so we discard it.

In [ ]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

If we now look at an element of our datasets, we will see the text have been replaced by the `input_ids` the model will need:

In [ ]:
tokenized_datasets["train"][1]

Now for the harder part: we need to concatenate all our texts together then split the result in small chunks of a certain `block_size`. To do this, we will use the `map` method again, with the option `batched=True`. This option actually lets us change the number of examples in the datasets by returning a different number of examples than we got. This way, we can create our new samples from a batch of examples.

First, we grab the maximum length our model was pretrained with. This might be a big too big to fit in your GPU RAM, so here we take a bit less at just 128.

In [ ]:
# block_size = tokenizer.model_max_length
block_size = 128

Then we write the preprocessing function that will group our texts:

In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

First note that we duplicate the inputs for our labels. This is because the model of the 🤗 Transformers library apply the shifting to the right, so we don't need to do it manually.

Also note that by default, the `map` method will send a batch of 1,000 examples to be treated by the preprocessing function. So here, we will drop the remainder to make the concatenated tokenized texts a multiple of `block_size` every 1,000 examples. You can adjust this behavior by passing a higher batch size (which will also be processed slower). You can also speed-up the preprocessing by using multiprocessing:

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

And we can check our datasets have changed: now the samples contain chunks of `block_size` contiguous tokens, potentially spanning over several of our original texts.

In [ ]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

In [ ]:
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16 # Use bfloat16 for faster computation
)

Now that the data has been cleaned, we're ready to instantiate our `Trainer`. We will a model:

In [ ]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    model_checkpoint,
    quantization_config=quantization_config,
    device_map="auto", # Automatically maps the model to the available devices (GPU)
    attn_implementation="eager"
)

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)

# This configures the LoRA adapter layers
lora_config = LoraConfig(
    r=16, # The rank of the update matrices. Lower rank means fewer trainable parameters.
    lora_alpha=32, # A scaling factor for the LoRA weights.
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"], # The modules to apply LoRA to.
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Wrap the model with PEFT
model = get_peft_model(model, lora_config)

# It's good practice to check the number of trainable parameters
model.print_trainable_parameters()


And some `TrainingArguments`:

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    output_dir=f"{model_name}-finetuned-wikitext2",
    per_device_train_batch_size=4,  # <-- Reduce batch size
    gradient_accumulation_steps=4, # <-- Use gradient accumulation
    learning_rate=2e-4,
    eval_strategy="epoch",
    logging_steps=100,
    num_train_epochs=1, # Adjust as needed
    weight_decay=0.01,
    push_to_hub=True, # Set to True if you want to push to Hugging Face Hub
    fp16=True, # Use mixed precision training
)

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/gpt-finetuned-wikitext2"` or `"huggingface/gpt-finetuned-wikitext2"`).

We pass along all of those to the `Trainer` class:

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

And we can train our model:

In [ ]:
trainer.train()

Once the training is completed, we can evaluate our model and get its perplexity on the validation set like this:

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
trainer.push_to_hub()

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("sgugger/my-awesome-model")
```

# Classifier Finetune

In [ ]:
# Install required libraries
!pip install -q -U transformers datasets accelerate peft bitsandbytes scikit-learn unsloth

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset, concatenate_datasets, DatasetDict

# --- 1. Load your .jsonl data directly using the datasets library ---
# Make sure to change the file paths to where you've stored your files.
relevant_path = '/content/drive/MyDrive/classifier_finetune/training_data_relevant_unique_500.jsonl'
irrelevant_path = '/content/drive/MyDrive/classifier_finetune/training_data_irrelevant_unique_500.jsonl'

# load_dataset with the 'json' type can handle .jsonl files automatically.
relevant_dataset = load_dataset('json', data_files=relevant_path, split='train')
irrelevant_dataset = load_dataset('json', data_files=irrelevant_path, split='train')

# --- 2. Combine, shuffle, and format the data ---
# Combine the two datasets into one
full_dataset = concatenate_datasets([relevant_dataset, irrelevant_dataset])

# Shuffle the combined dataset to mix relevant and irrelevant samples
full_dataset = full_dataset.shuffle(seed=42)

# This function converts one of your JSON objects into a single string.
# (This function is the same as before)
def create_prompt(sample):
    prompt = "### INSTRUCTION\n"
    prompt += "Classify the user's message as RELEVANT or IRRELEVANT to the math problem and conversation history.\n\n"

    # The data is now nested under a 'context' key, so we access it like sample['context']['problem']
    prompt += "### MATH PROBLEM\n"
    prompt += f"Problem: {sample['context']['problem']}\n"
    prompt += f"Solution: {sample['context']['solution']}\n\n"

    prompt += "### CONVERSATION HISTORY\n"
    for message in sample['history']:
        role = "AI Tutor" if message['role'] == 'ai' else "Student"
        prompt += f"{role}: {message['content']}\n"
    prompt += "\n"

    prompt += "### USER MESSAGE\n"
    prompt += f"{sample['user_message']}\n\n"

    prompt += "### CLASSIFICATION\n"
    return prompt

# Apply the formatting function to create a new 'prompt' column
# We keep the 'label' column for the next step.
def format_dataset(example):
    example['prompt'] = create_prompt(example)
    return example

full_dataset = full_dataset.map(format_dataset)

# --- 3. Split the data into training and testing sets ---
# An 80/20 or 90/10 split is common.
final_datasets = full_dataset.train_test_split(test_size=0.1)

print("Training set size:", len(final_datasets['train']))
print("Test set size:", len(final_datasets['test']))
print("\nExample of a formatted prompt:")
print(final_datasets['train'][0]['prompt'])
print("\nExample label:", final_datasets['train'][0]['label'])

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "google/gemma-3-1b-it"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
print(tokenizer.pad_token)

def preprocess_function(samples):
    inputs = []
    for prompt, label in zip(samples['prompt'], samples['label']):
        inputs.append(prompt + label)

    model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding="longest")
    model_inputs["labels"] = model_inputs["input_ids"].copy()

    for i, prompt in enumerate(samples['prompt']):
        prompt_ids = tokenizer(prompt, max_length=1024, truncation=True)["input_ids"]
        model_inputs["labels"][i][:len(prompt_ids)] = [-100] * len(prompt_ids)

    return model_inputs

tokenized_datasets = final_datasets.map(preprocess_function, batched=True, remove_columns=final_datasets['train'].column_names)

In [ ]:
# --- The Final, Corrected Preprocessing Function ---

from transformers import AutoTokenizer
import numpy as np # Make sure to import numpy

model_checkpoint = "google/gemma-3-1b-it"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token

def preprocess_function(samples):
    inputs = [prompt + label for prompt, label in zip(samples['prompt'], samples['label'])]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="longest")
    labels_batch = np.array(model_inputs["input_ids"])
    padding_mask = np.array(model_inputs["attention_mask"]) == 0
    labels_batch[padding_mask] = -100
    for i, prompt in enumerate(samples['prompt']):
        prompt_ids = tokenizer(prompt, max_length=512, truncation=True)["input_ids"]
        prompt_len = len(prompt_ids)
        labels_batch[i, :prompt_len] = -100
    model_inputs["labels"] = labels_batch.tolist()
    return model_inputs

tokenized_datasets = final_datasets.map(preprocess_function, batched=True, remove_columns=final_datasets['train'].column_names)

In [ ]:
from datasets import load_dataset, concatenate_datasets, DatasetDict
from transformers import AutoTokenizer
import numpy as np

# --- 1. Load your .jsonl data ---
# This part remains the same.
relevant_path = '/content/drive/MyDrive/classifier_finetune/training_data_relevant_unique_500.jsonl'
irrelevant_path = '/content/drive/MyDrive/classifier_finetune/training_data_irrelevant_unique_500.jsonl'

relevant_dataset = load_dataset('json', data_files=relevant_path, split='train')
irrelevant_dataset = load_dataset('json', data_files=irrelevant_path, split='train')

# --- 2. Combine and shuffle the data ---
# This part remains the same.
full_dataset = concatenate_datasets([relevant_dataset, irrelevant_dataset])
full_dataset = full_dataset.shuffle(seed=42)

# --- 3. Split the data into training and testing sets ---
# We split the raw data first.
final_datasets = full_dataset.train_test_split(test_size=0.1)


# --- 4. Load the tokenizer ---
# The tokenizer is needed to apply the model's specific chat template.
# Note: This will be loaded again by Unsloth, which is fine. We need it here for formatting.
model_checkpoint = "google/gemma-3-1b-it"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


# --- 5. Format the dataset using the official chat template ---
# This single function replaces your old create_prompt, format_dataset,
# and preprocess_function.
def format_chat_template(example):
    # The history and context are formatted as a single "user" turn.
    messages = [
        {
            "role": "user",
            "content": (
                "Classify the user's message as RELEVANT or IRRELEVANT to the math problem and conversation history.\n\n"
                "### MATH PROBLEM\n"
                f"Problem: {example['context']['problem']}\n"
                f"Solution: {example['context']['solution']}\n\n"
                "### CONVERSATION HISTORY\n" +
                "".join(
                    f"{'AI Tutor' if msg['role'] == 'ai' else 'Student'}: {msg['content']}\n"
                    for msg in example['history']
                ) + "\n"
                "### USER MESSAGE\n"
                f"{example['user_message']}"
            )
        },
        # The assistant's label is the response it is being taught to generate.
        {
            "role": "assistant",
            "content": example['label']
        }
    ]
    # The tokenizer handles adding all the special tokens like '<start_of_turn>' and '<end_of_turn>'.
    # We create a new column called "text" which contains the fully formatted string.
    return {"text": tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)}

# Apply the new formatting function to our datasets.
# Unsloth will handle the tokenization internally during training.
formatted_datasets = final_datasets.map(format_chat_template)

# --- 6. Verify the new format ---
# We print the output to see the result of the new formatting.
print("Training set size:", len(formatted_datasets['train']))
print("Test set size:", len(formatted_datasets['test']))
print("\nExample of a formatted prompt from the 'text' column:")
print(formatted_datasets['train'][0]['text'])

# The 'formatted_datasets' object is now ready to be passed directly into the Trainer.
# Make sure to use dataset_text_field="text" in your Trainer arguments.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def analyze_token_lengths(dataset, tokenizer, text_column="text"):
    """
    Calculates, prints statistics, and plots the distribution of token lengths
    for a given Hugging Face dataset.

    Args:
        dataset (datasets.Dataset): The dataset to analyze (e.g., formatted_datasets['train']).
        tokenizer (transformers.PreTrainedTokenizer): The tokenizer for converting text to tokens.
        text_column (str): The name of the column in the dataset that contains the text.
    """
    # Use the efficient .map() function to tokenize and get the length of each input
    # This is much faster than iterating through the dataset in a loop.
    token_lengths = dataset.map(
        lambda example: {'length': len(tokenizer(example[text_column]).input_ids)},
        num_proc=4  # Use multiple processes to speed up the calculation
    )['length']

    token_lengths = np.array(token_lengths)

    # --- Calculate and Print Statistics ---
    print("--- Token Length Analysis ---")
    print(f"Total examples: {len(token_lengths)}")
    print(f"Min token length:    {np.min(token_lengths)}")
    print(f"Max token length:    {np.max(token_lengths)}")
    print(f"Mean token length:   {np.mean(token_lengths):.2f}")
    print(f"Median token length: {np.median(token_lengths)}")
    print("\n--- Percentiles ---")
    print(f"90th percentile: {np.percentile(token_lengths, 90):.0f}")
    print(f"95th percentile: {np.percentile(token_lengths, 95):.0f}")
    print(f"99th percentile: {np.percentile(token_lengths, 99):.0f}")
    print("-" * 29)
    print(f"\nThis analysis helps you choose an optimal `max_seq_length`.")
    print(f"Your current max_seq_length is set to 512.")
    print(f"Examples longer than this will be truncated.")
    print("-" * 29)


    # --- Plot the Histogram ---
    plt.figure(figsize=(12, 6))
    plt.hist(token_lengths, bins=50, alpha=0.75, color='royalblue', edgecolor='black')
    plt.axvline(np.mean(token_lengths), color='red', linestyle='dashed', linewidth=2, label=f'Mean: {np.mean(token_lengths):.2f}')
    plt.axvline(np.percentile(token_lengths, 95), color='orange', linestyle='dashed', linewidth=2, label=f'95th Percentile: {np.percentile(token_lengths, 95):.0f}')
    plt.title('Distribution of Input Token Lengths')
    plt.xlabel('Token Length')
    plt.ylabel('Frequency (Number of Examples)')
    plt.legend()
    plt.grid(axis='y', alpha=0.75)
    plt.show()


# --- Example of how to call the function ---
# Make sure this is placed AFTER you have created your `formatted_datasets`
# and loaded your `tokenizer`.

print("Analyzing the training set:")
analyze_token_lengths(formatted_datasets['train'], tokenizer)

print("\nAnalyzing the test set:")
analyze_token_lengths(formatted_datasets['test'], tokenizer)

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


def preprocess_logits_for_metrics(logits, labels):
    if isinstance(logits, tuple):
        logits = logits[0]
    return logits.argmax(dim=-1)

# Using your preferred, explicit metric calculation
def compute_metrics(eval_pred):
    preds, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels[labels == -100] = tokenizer.pad_token_id
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]
    decoded_preds = [pred if pred else "__empty__" for pred in decoded_preds] # Handle empty predictions

    # Use the individual score functions as you requested
    precision = precision_score(decoded_labels, decoded_preds, average='binary', pos_label='RELEVANT', zero_division=0)
    recall = recall_score(decoded_labels, decoded_preds, average='binary', pos_label='RELEVANT', zero_division=0)
    f1 = f1_score(decoded_labels, decoded_preds, average='binary', pos_label='RELEVANT', zero_division=0)
    acc = accuracy_score(decoded_labels, decoded_preds)

    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}

In [ ]:
import torch
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# QLoRA configuration
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load the quantized model
model = AutoModelForCausalLM.from_pretrained(
    model_checkpoint,
    quantization_config=quantization_config,
    device_map="auto",
    attn_implementation="eager"
)

# Make sure the model knows the new pad token
model.resize_token_embeddings(len(tokenizer))

# Prepare model for PEFT
model = prepare_model_for_kbit_training(model)

# LoRA configuration
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

model_name = model_checkpoint.split("/")[-1]
# Training Arguments
training_args = TrainingArguments(
    output_dir=f"{model_name}-classifier-finetune",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    learning_rate=2e-4,
    num_train_epochs=3, # You can start with 3-5 epochs for a small dataset
    logging_steps=25,
    gradient_checkpointing=True,
    optim="paged_adamw_8bit",
    fp16=True,
    push_to_hub=True, # Set to True if you want to push to Hugging Face Hub
    # Evaluation settings
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics, # Pass in our new metrics function
)

In [ ]:
import torch
import unsloth
import numpy as np
from transformers import AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset, concatenate_datasets
from trl import SFTTrainer


model, tokenizer = unsloth.FastLanguageModel.from_pretrained(
    model_name = model_checkpoint,
    max_seq_length = 512,
    dtype = None, # Will auto-detect torch.bfloat16 if available
    load_in_4bit = True,
)

model = unsloth.FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha = 32,
    lora_dropout = 0.05,
    bias = "none",
    task_type = "CAUSAL_LM",
)
model.print_trainable_parameters()

model_name = model_checkpoint.split("/")[-1]
# Training Arguments
training_args = TrainingArguments(
    output_dir=f"{model_name}-classifier-finetune",
    num_train_epochs=3,
    per_device_train_batch_size = 1,  # We can likely go back to 2 now!
    gradient_accumulation_steps = 16, # Effective batch size is still 16
    gradient_checkpointing = True,   # Keep this for extra safety
    optim="adamw_8bit", # Unsloth works great with 8bit optimizers
    learning_rate=2e-4,
    logging_steps=25,
    fp16 = not torch.cuda.is_bf16_supported(), # Use bf16 if available, else fp16
    bf16 = torch.cuda.is_bf16_supported(),
    push_to_hub=True,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=formatted_datasets["train"],
    eval_dataset=formatted_datasets["test"],
    dataset_text_field="text",
    max_seq_length=512,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
)

In [ ]:
# Start training!
trainer.train()